# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Huỳnh Đức Thiện
- MSSV: 21127693
- Lớp: 21CLC07

# Import

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

# Đọc dữ liệu

In [ ]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu huấn luyện)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Xử lý dữ liệu

In [ ]:
# before
plt.rcParams['figure.figsize'] = (22, 10)

plt.subplot(1, 2, 1)
plt.title('Salary Distribution Plot')
sns.histplot(train['Salary'])
plt.xlabel('Salary')
plt.ylabel('Density')

plt.subplot(1, 2, 2)
plt.title('Salary Spread')
sns.boxplot(y=train['Salary'])
plt.ylabel('Salary')

plt.show()

# data processing
train = train.dropna()

median = train['Salary'].median()
mad = np.median(np.abs(train['Salary'] - median))
modified_z_scores = 0.6745 * (train['Salary'] - median) / mad

threshold = 3.5

outliers = train[modified_z_scores.abs() > threshold]

train = train[modified_z_scores.abs() <= threshold]

# after
plt.rcParams['figure.figsize'] = (22, 10)

plt.subplot(1, 2, 1)
plt.title('Salary Distribution Plot (Outliers Removed)')
sns.histplot(train['Salary'])
plt.xlabel('Salary')
plt.ylabel('Density')

plt.subplot(1, 2, 2)
plt.title('Salary Spread (Outliers Removed)')
sns.boxplot(y=train['Salary'])
plt.ylabel('Salary')

plt.show()

# Cài đặt hàm

In [ ]:
# Cài đặt các hàm cần thiết ở đây

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [ ]:
# Phần code cho yêu cầu 1a

selected_features = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain']

X_train_1a = train[selected_features]
y_train_1a = train['Salary']

X_test_1a = test[selected_features]
y_test_1a = test['Salary']

model = LinearRegression()
model.fit(X_train_1a, y_train_1a)

intercept = model.intercept_
coefficients = model.coef_

formula = f"Salary = {intercept:.3f}"

for feature, coefficient in zip(selected_features, coefficients):
    formula += f" + {coefficient:.3f} * {feature}"

print("Regression model formula:\n")
print(formula)

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra

y_test_pred = model.predict(X_test_1a)

mae = mean_absolute_error(y_test_1a, y_test_pred)
print("Mean Absolute Error (MAE):", mae)

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [ ]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
best_feature = None
best_mae = None

train_shuffled = train.sample(frac=1, random_state=42)

selected_features = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']

for ft in selected_features:
    X_train_1b = train_shuffled[[ft]]
    y_train_1b = train_shuffled['Salary']
    
    model = LinearRegression()
    
    mae_scores = -cross_val_score(model, X_train_1b, y_train_1b, scoring='neg_mean_absolute_error', cv=5)
    cur_mae = mae_scores.mean()
    
    if best_mae is None or cur_mae < best_mae:
        best_feature = ft
        best_mae = cur_mae
    
    print(f"MAE for feature '{ft}': {cur_mae}")

print(f"\nBest Feature: {best_feature}")
print(f"MAE for Best Feature: {best_mae}")

In [ ]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

X_train_best = train[[best_feature]]
y_train_best = train['Salary']

best_personality_feature_model = LinearRegression()
best_personality_feature_model.fit(X_train_best, y_train_best)

intercept_best = best_personality_feature_model.intercept_
coefficient_best = best_personality_feature_model.coef_[0]

formula_best = f"Salary = {intercept_best:.3f} + {coefficient_best:.3f} * {best_feature}"
print("\nRegression model formula for the best feature:\n")
print(formula_best)

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_test_best = test[[best_feature]]
y_test_pred_best = best_personality_feature_model.predict(X_test_best)
mae_best = mean_absolute_error(y_test, y_test_pred_best)
print("\nMean Absolute Error (MAE) for the best feature:", mae_best)

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [ ]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

selected_features = ['English', 'Logical', 'Quant']

for ft in selected_features:
    X_train_1c = train_shuffled[[ft]]
    y_train_1c = train_shuffled['Salary']
    
    model = LinearRegression()
    
    mae_scores = -cross_val_score(model, X_train_1c, y_train_1c, scoring='neg_mean_absolute_error', cv=5)
    cur_mae = mae_scores.mean()
    
    if best_mae is None or cur_mae < best_mae:
        best_feature = ft
        best_mae = cur_mae
    
    print(f"MAE for feature '{ft}': {cur_mae}")

print(f"\nBest Feature: {best_feature}")
print(f"MAE for Best Feature: {best_mae}")

In [ ]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

X_train_best = train[[best_feature]]
y_train_best = train['Salary']

best_skill_feature_model = LinearRegression()
best_skill_feature_model.fit(X_train_best, y_train_best)

intercept_best = best_skill_feature_model.intercept_
coefficient_best = best_skill_feature_model.coef_[0]

formula_best = f"Salary = {intercept_best:.3f} + {coefficient_best:.3f} * {best_feature}"
print("\nRegression model formula for the best feature:\n")
print(formula_best)

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
X_test_best = test[[best_feature]]
y_test_pred_best = best_skill_feature_model.predict(X_test_best)
mae_best = mean_absolute_error(y_test, y_test_pred_best)
print("\nMean Absolute Error (MAE) for the best feature:", mae_best)

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [ ]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [ ]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [ ]:
# Option 1 - Find the top 10 features to use for training the model.
all_features = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain', 'ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg', 'conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']
mae_features = []

for feature in all_features:
    X_train = train[[feature]]
    y_train = train['Salary']

    model = LinearRegression()
    mae_scores = -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    mae = mae_scores.mean()

    mae_features.append((feature, mae))

mae_features.sort(key=lambda x: x[1])
top_features = [feature for feature, _ in mae_features[:10]]

print("Top 10 Features:")
for feature, mae in mae_features[:10]:
    print(f"{feature} -- {mae}")

X_train = train[top_features]
y_train = train['Salary']
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'\nMAE for dataframe with top 10 features: {mae}')

In [ ]:
# Option 2 - Through data visualization, identify features that exhibit a tendency to correlate with the target value.
for feature in all_features:
    plt.scatter(train[feature], train['Salary'])
    plt.xlabel(feature)
    plt.ylabel('Salary')
    plt.title(f'Scatter plot of {feature} vs Salary')
    plt.show()

In [ ]:
selected_features = ['10percentage', '12percentage', 'collegeGPA', 'English', 'Logical', 'Quant', 'conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']

X_train = train_shuffled[selected_features]
y_train = train_shuffled['Salary']

model = LinearRegression()
model.fit(X_train, y_train)

mae_scores = -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
cur_mae = mae_scores.mean()

print(f"MAE for dataframe with visualized-selected features: {cur_mae}")

In [ ]:
# Option 3 - Implement additional new features derived from existing features.
modified_train = train.copy()

modified_train['engineering_skills'] = modified_train[['ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg']].mean(axis=1)
modified_train['collegeGPA_squared'] = modified_train['collegeGPA'] ** 2
modified_train['collegeGPA_cube'] = modified_train['collegeGPA'] ** 3
modified_train['interaction_percentage'] = modified_train['10percentage'] * modified_train['12percentage']
modified_train['10percentage_bins'] = pd.cut(modified_train['10percentage'], bins=[0, 50, 75, 100], labels=['low', 'medium', 'high'])
modified_train['12percentage_bins'] = pd.cut(modified_train['12percentage'], bins=[0, 50, 75, 100], labels=['low', 'medium', 'high'])

X_train = modified_train.drop('Salary', axis=1)
y_train = modified_train['Salary']

model = LinearRegression()

X_train_encoded = pd.get_dummies(X_train, columns=['10percentage_bins', '12percentage_bins'])
mae_scores = -cross_val_score(model, X_train_encoded, y_train, scoring='neg_mean_absolute_error', cv=5)
cur_mae = mae_scores.mean()

print(f"MAE for dataframe with additional features: {cur_mae}")


In [ ]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

my_best_model = LinearRegression()
my_best_model.fit(X_train_encoded, y_train)

selected_features = list(X_train_encoded.columns)  
intercept = my_best_model.intercept_
coefficients = my_best_model.coef_

formula = f"Salary = {intercept:.3f}"

for feature, coefficient in zip(selected_features, coefficients):
    formula += f" + {coefficient:.3f} * {feature}"

print("Regression model formula:\n")
print(formula)

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
modified_test = test.copy()

modified_test['engineering_skills'] = modified_test[['ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg']].mean(axis=1)
modified_test['collegeGPA_squared'] = modified_test['collegeGPA'] ** 2
modified_test['collegeGPA_cube'] = modified_test['collegeGPA'] ** 3
modified_test['interaction_percentage'] = modified_test['10percentage'] * modified_test['12percentage']
modified_test['10percentage_bins'] = pd.cut(modified_test['10percentage'], bins=[0, 50, 75, 100], labels=['low', 'medium', 'high'])
modified_test['12percentage_bins'] = pd.cut(modified_test['12percentage'], bins=[0, 50, 75, 100], labels=['low', 'medium', 'high'])

X_test = modified_test.drop('Salary', axis=1)
X_test_encoded = pd.get_dummies(X_test, columns=['10percentage_bins', '12percentage_bins'])

y_test_pred = my_best_model.predict(X_test_encoded)
mae = mean_absolute_error(y_test, y_test_pred)
print("\nMean Absolute Error (MAE) for the test dataset with my_best_model:", mae)

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$